Krishna Sharma AP22110010128

In [15]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import re

In [3]:
df = pd.read_csv('iris_with_missing_cells.csv')
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,NaN,3.5,1.4,0.2,0.0
1,4.9,NaN,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,NaN,3.1,1.5,0.2,0.0
4,NaN,3.6,1.4,NaN,0.0


In [4]:
df.isnull().sum()

sepal length (cm)    25
sepal width (cm)     14
petal length (cm)    21
petal width (cm)     27
target               14
dtype: int64

In [5]:
# overall percentage of missing values
print((df.isnull().sum().sum() / (df.size)) * 100,'%')

13.466666666666665 %


In [6]:
# feature wise percentage of total data points missing
(df.isnull().sum()/(len(df)))*100

sepal length (cm)    16.666667
sepal width (cm)      9.333333
petal length (cm)    14.000000
petal width (cm)     18.000000
target                9.333333
dtype: float64

In [7]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,125.000000,136.000000,129.000000,123.000000,136.000000
mean,5.853600,3.056618,3.732558,1.202439,1.007353
std,0.794593,0.441664,1.781904,0.769017,0.811914
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.500000,0.300000,0.000000
50%,5.800000,3.000000,4.300000,1.300000,1.000000
75%,6.400000,3.400000,5.100000,1.800000,2.000000
max,7.700000,4.400000,6.900000,2.500000,2.000000


In [8]:
df_mean = df.copy()
for column in df_mean.select_dtypes(include=[np.number]).columns:
    mean = df_mean[column].mean()
    df_mean[column].fillna(mean, inplace=True)      
    
df_mean.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.853600,3.056618,3.732558,1.202439,1.007353
std,0.724874,0.420403,1.651567,0.695862,0.772830
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,5.400000,2.800000,1.600000,0.400000,0.000000
50%,5.853600,3.028309,4.000000,1.202439,1.000000
75%,6.300000,3.300000,4.975000,1.700000,2.000000
max,7.700000,4.400000,6.900000,2.500000,2.000000


In [9]:
df_class_wise_mean = df.copy()
numeric_cols = df_class_wise_mean.select_dtypes(include=[np.number]).columns
if 'species' in df_class_wise_mean.columns:
    for col in numeric_cols:
        df_class_wise_mean[col] = df_class_wise_mean.groupby('species')[col].apply(lambda x: x.fillna(x.mean()))

df_class_wise_mean.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,125.000000,136.000000,129.000000,123.000000,136.000000
mean,5.853600,3.056618,3.732558,1.202439,1.007353
std,0.794593,0.441664,1.781904,0.769017,0.811914
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.500000,0.300000,0.000000
50%,5.800000,3.000000,4.300000,1.300000,1.000000
75%,6.400000,3.400000,5.100000,1.800000,2.000000
max,7.700000,4.400000,6.900000,2.500000,2.000000


In [10]:
df_knn = df.copy()
numeric_cols = df_knn.select_dtypes(include=[np.number]).columns

for index, row in df_knn.iterrows():
    for col in numeric_cols:
        if pd.isnull(row[col]):
            distances = []
            for i, r in df_knn.iterrows():
                if i != index and not pd.isnull(r[col]):
                    dist = np.linalg.norm(df_knn.loc[index, numeric_cols].dropna() - df_knn.loc[i, numeric_cols].dropna())
                    distances.append((dist, r[col]))
            
            distances.sort(key=lambda x: x[0])
            knn_mean = np.mean([val for _, val in distances[:3]])
            df_knn.at[index, col] = knn_mean

df_knn.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.690000,3.081037,3.401333,1.022000,0.913333
std,0.812576,0.427305,1.845570,0.795952,0.826870
min,4.400000,2.000000,1.000000,0.100000,0.000000
25%,4.875000,2.800000,1.400000,0.200000,0.000000
50%,5.700000,3.050000,4.000000,1.100000,1.000000
75%,6.300000,3.322222,4.975000,1.700000,2.000000
max,7.700000,4.400000,6.900000,2.500000,2.000000


In [56]:
DOB = ['01/15/1992', 'March 5, 1985', '1987-07-23', '7th August 1990', '04-09-1978', 'Dec 11, 2001', '1995/10/21', '22nd May, 1983', '12.03.1999', 
       '11/07/1984', '1986-Aug-14', '2000 06 17', '09-April-1993', 'May 31st, 1997', '1999.12.25', '2nd Feb 1988', '07-04-2002', 'January 9, 1991', 
       '17/03/1986', 'Oct-19-1976', '1994/05/30', '05.11.2003', '1998 Mar 22', '4th-July-1980', '21.06.1990', '09_12_1985', '14-Aug-1999', 
       '1982 December 8', '06.05.1987', 'Sep 27, 2005', '2001/11/03', '1996_07_18', 'Mar 14th 2004', '15-09-1992', '1985/04/05', '10 02 1979', 
       '8th June, 2008', '07_21_1993', '2000-Nov-26', 'June 12 1984', '25.07.1989', '3rd-Oct-1995', '2002-03-08', '08/09/1997', 'Nov 29, 1981', 
       '04/Apr/2003', '1975-01-18', '16 08 1986', 'Sep_14_1994', '1989-December-03']

In [57]:
month_map = {
    'January': '01', 'February': '02', 'March': '03', 'April': '04', 'May': '05', 'June': '06', 
    'July': '07', 'August': '08', 'September': '09', 'October': '10', 'November': '11', 'December': '12',
    'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 
    'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
}

In [75]:
def convert(dob):
    dob = re.sub(r'(\d+)(st|nd|rd|th)', r'\1', dob)
    dob = re.sub(r'[_/,.\-]', ' ', dob)

    parts = dob.split()
    day, month, year = None, None, None
    
    for part in parts:
        if part.isdigit():
            if len(part) == 4:  
                year = part  
            elif int(part) > 31:  
                year = part  
            elif int(part) <= 12 and not month:
                month = part  
            else:
                day = part 
        elif part in month_map:
            month = month_map[part]

    if not year:
        year = parts[-1]  
    if not month:
        month = '01'  
    if not day:
        day = '01'  

    return year + "-" + month.zfill(2) + "-" + day.zfill(2)

In [76]:
for dob in DOB:
    DOB_converted = convert(dob)

df = pd.DataFrame({'Original DOB': DOB, 'Converted DOB': DOB_converted})
df

,Original DOB,Converted DOB
0,01/15/1992,1989-12-03
1,"March 5, 1985",1989-12-03
2,1987-07-23,1989-12-03
3,7th August 1990,1989-12-03
4,04-09-1978,1989-12-03
5,"Dec 11, 2001",1989-12-03
6,1995/10/21,1989-12-03
7,"22nd May, 1983",1989-12-03
8,12.03.1999,1989-12-03
9,11/07/1984,1989-12-03


Krishna Sharma AP22110010128